# **DEMO - RUSLE a partir de datos satelitales**
## Diplomado evaluación ambiental y territorial del recurso suelo

Esta demostración tiene como objetivo mostrar el flujo de trabajo al usar datos remotos, desde reunir los datos desde diferentes fuentes, llevar a cabo los preprocesos necesarios, análisis, visualizar y exportar los datos.

Para esto se calculará la perdida de suelo anual (ton/ha/año) utilizando el modelo empírico [RUSLE (Revised Universal Soil Loss Equation)](https://www.eea.europa.eu/data-and-maps/figures/rusle-soil-erosion-model-structure/rusle-soil-erosion-model-structure).

Todo el flujo de trabajo se llevará a cabo en el ambiente de [Google Colaboratory](https://www.androidpolice.com/google-colab-explainer/).

Utilizando el lenguaje de programación Python y la [API de Google Earth Engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api).

Para esto usaremos como entradas exclusivamente datos satelitales. Descritos a continuación:

1. **Precipitación**

  Los datos de precipitación provienen del modelo [Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0)](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY). Este provee datos diarios de precipitación a nivel quasi global y es generado utilizando diferentes datos satelitales y estaciones meteorológicas ([Funk et al., 2015](https://www.nature.com/articles/sdata201566)).

  Se eligió este modelo sobre otros modelos satelitales o incluso modelos generados por grupos en Chile (como CR2Met ([Boisier et al., 2018](https://ui.adsabs.harvard.edu/abs/2018EGUGA..2019739B%2F/abstract))) principalmente porque la serie de tiempo es más larga, lo que nos permite llevar a cabo el análisis en más años y ya que se ha concluido que funciona bien respecto a estaciones meteorológicas desde en centro hacia el sur de Chile ([Zambrano et al., 2017](https://www.sciencedirect.com/science/article/pii/S0169809516305865)) como se puede observar en la figura 1.

  Con los datos de precipitación podemos calcular el factor R del modelo RUSLE.

![](https://drive.google.com/uc?export=view&id=1ELvpcuAcv9ufXJ2AjMOzC8WXEtIPsEAw)

Figura 1. Boxplots precipitación in situ contra differentes productos satelitales ([Zambrano et al., 2017](https://www.sciencedirect.com/science/article/pii/S0169809516305865)).

2. **Suelos**

  Los datos de suelo provienen de la base de datos de   [OpenLandMaps](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_v02) alojada en el catalogo de Google Earth Engine. Estos corresponden a la clase textural de los primeros 10 cm de suelo.

  Con los datos de suelo podemos calcular el factor K del modelo RUSLE.

3. **Modelo digital de elevación (DEM)**

  El DEM utilizado corresponde a [Shuttle Radar Topography Mission (SRTM)](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4) el cuál fue descargado desde el catalogo de Google Earth Engine con una resolución espacial de 90 m.

  Con el DEM podemos estimar los factores L y S del modelo RUSLE.

4. **Vigor fotosintetico**

  Los datos asociados al vigor fotosintetico corresponden al NDVI calculado utilizando images de la serie de [ reflectancia de superficie landsat 8 Tier 1](https://developers.google.com/earth-engine/datasets/catalog/landsat-8). Para esto se carga la serie completa del satelite, se seleccionan las bandas de interés (NIR y RED) y se calcula el indice de vegetación.

  Con los datos de vigor fotosintetico podemos estimar el factor C del modelo RUSLE.

5. **Cobertura de suelo**

  Los datos de cobertura de suelo corresponden al "Landcover" de [MODIS](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1) el cual ofrece un mapa de cobertura de suelos en 500 m de resolución espacial una vez por año.

Finalmente se calculará la perdida de suelo a nivel de cuenca posterior a los incendios del año 2017 para comparar como las diferencias en la cobertura de suelo modifican la erosión a nivel de paisaje.

#### Modelo Rusle:


$A = R * K * L * S * C * P$

Donde:

**A:** Perdida de suelo (ton/ha/año) \
**R:** Erosividad producidad por la lluvia \
**K:** Erodabilidad de suelo \
**L:** Longitud de la pendiente \
**S:** Inclinación de la pendiente \
**C:** Cobertura vegetal \
**P:** Practicas de control de la erosión \

# 0. Cargar librerias y API

In [ ]:
!pip install geemap
!pip install importlib-metadata==4.0.1
!pip install xarray==0.18.1

import ee
import geemap
import math
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
#### Configuración
date_from = '2014-01-01'
date_to = '2015-01-01'

# Poligono de cuenca rio Purapel descargado desde CAMELS-CL (https://camels.cr2.cl/)
aoi = geemap.geojson_to_ee('https://github.com/LRiveroIribarne/RUSLE_Satelital_DEMO_EATS/blob/fb370ac9cfea5f5f57e7d0861487d6b935364c0b/data/aoi.geojson')
aoi

Downloading...
From: https://raw.githubusercontent.com/LRiveroIribarne/RUSLE_Satelital_DEMO_EATS/fb370ac9cfea5f5f57e7d0861487d6b935364c0b/data/aoi.geojson
To: /tmp/87683d6f-1dbc-4b34-9925-3f1ba17e464a.geojson
159kB [00:00, 47.8MB/s]                    


## 1. Colección de datos

In [ ]:
# Precipitacón CHIRPS - https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY
pcp = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

Map = geemap.Map()

precipitationVis = {
  'min': 1.0,
  'max': 20.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

Map.addLayer(pcp.first(), vis_params=precipitationVis, name='Precipitación CHIRPS')
Map.add_colorbar(precipitationVis, label="Precipitación", layer_name="Precipitación CHIRPS")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Suelos
# Soil texture class (USDA system) at 0 cm depth
soils = ee.Image('OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02')

Map = geemap.Map()
Map.addLayer(soils.select('b0'))

legend_dict = {
    '1 Cl': 'd5c36b',
    '2 SiCl': 'b96947',
    '3 SaCl': '9d3706',
    '4 ClLo': 'ae868f',
    '5 SiClLo': 'f86714',
    '6 SaClLo': '46d143',
    '7 Lo': '368f20',
    '8 SiLo': '3e5a14',
    '9 SaLo': 'ffd557',
    '10 Si': 'fff72e',
    '11 LoSa': 'ff5a9d',
    '12 Sa': 'ff005b',
}

Map.add_legend(legend_title="Clase textural", legend_dict=legend_dict)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Modelo digital de elevación (DEM)
SRTM_dem = ee.Image('USGS/SRTMGL1_003')

DEMVis = {
  'min': 0.0,
  'max': 6000,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

colors = DEMVis['palette']
vmin = DEMVis['min']
vmax = DEMVis['max']

Map = geemap.Map()
Map.addLayer(SRTM_dem,  vis_params=DEMVis, name="SRTM")
Map.add_colorbar(DEMVis, label="Height", layer_name="SRTM")

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Landsat 8
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

Map = geemap.Map()

vizParams = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 15000,
  'gamma': [0.95, 1.1, 1]
}

Map.addLayer(l8.median(), vis_params=vizParams)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# MODIS
modis = ee.ImageCollection("MODIS/006/MCD12Q1").select('LC_Type1')

Map = geemap.Map()

igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.addLayer(modis, vis_params=igbpLandCoverVis, name='MODIS Land Cover')
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## 2. Factores RUSLE

#### 2.1 Erosibilidad producida por la lluvia (R)

La erosibilidad por parte de la lluvia es la estimación de la perdida de suelo a causa de la precipitación (Uddin et al. 2018).

$R = 0.0483 * P^{1.610}$

Donde P corresponde a la precipitación anual

In [ ]:
pcp_filtered = pcp.select('precipitation').filterDate(date_from, date_to).sum().clip(aoi)

Map = geemap.Map()

precipitationVis = {
  'min': 0.0,
  'max': 1000.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}

colors = precipitationVis['palette']
vmin = precipitationVis['min']
vmax = precipitationVis['max']

Map.addLayer(pcp_filtered, vis_params=precipitationVis, name='Precipitación CHIRPS')
Map.add_colorbar(precipitationVis, label="Precipitación", layer_name="Precipitación CHIRPS")

Map.centerObject(aoi, 10)
Map

Map(center=[-35.544278357280355, -72.13937168162737], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# R factor
R_factor2014 = ee.Image(pcp_filtered.pow(1.610).multiply(0.0483))

#### 2.2 Erodavilidad de suelo (K)

La erodavilidad de suelo se asocia al tipo de suelo (en este ejercicio, asociado a la clase textural de los primeros 10 cm de suelo). Indica la susceptibilidad del suelo a ser erosionado por la lluvia y la escorrentia superficial.


In [ ]:
soils_filtered = soils.select('b0').clip(aoi).rename('soil')

In [ ]:
# Valores obtenidos desde: https://github.com/sukantjain/Google-Earth-Engine/blob/main/JavaScript/RUSLE%20in%20GEE.js

K_factor2014 = soils_filtered.expression(
    "(b('soil') == 11) ? 0.080" +                # Sand
      ": (b('soil') == 10) ? 0.040" +            # Loamy Sand
        ": (b('soil') == 9) ? 0.300" +           # Silt
           ": (b('soil') == 8) ? 0.070" +         # Sandy Loam
            ": (b('soil') == 7) ? 0.380" +       # Silt Loam
            ": (b('soil') == 6) ? 0.300" +       # Loam
            ": (b('soil') == 5) ? 0.350" +       # Sandy Clay Loam
            ": (b('soil') == 4) ? 0.300" +       # Silty Clay Loam
            ": (b('soil') == 3) ? 0.300" +       # Clay Loam
            ": (b('soil') == 2) ? 0.220" +        # Sandy Clay
            ": (b('soil') == 1) ? 0.260" +       # Silty Clay
            ": (b('soil') == 0) ? 0.220" +       # Clay
             ": 9999").rename('K').clip(aoi)

In [ ]:
# LA totalidad de la cuenca es 9 o 6 -> 0.300
Map = geemap.Map()
Map.addLayer(K_factor2014.select('K'))
Map.centerObject(aoi, 10)
Map

Map(center=[-35.544278357280355, -72.13937168162737], controls=(WidgetControl(options=['position', 'transparen…

#### 2.3 Longitud de la pendiente (L)

La longitud de la pendiente representa la distancia desde un pixel dado hasta hasta el punto de erosión potencial.

$L = (\frac{λ}{22.13})^m$

Donde λ corresponde a la longitud de la pendiente y *m* toma un valor entre 0.2 y 0.5en función de la pendiente.

In [ ]:
elev = SRTM_dem.select('elevation')
slope_deg = ee.Terrain.slope(elev).clip(aoi)
slope_perc = slope_deg.divide(180).multiply(math.pi).tan().multiply(100) # slope from degrees to percentage

m = ee.Image(1).where(slope_perc.gt(-1).And(slope_perc.lte(1)), 0.2).where(slope_perc.gt(1).And(slope_perc.lte(3)), 0.3).where(slope_perc.gt(3).And(slope_perc.lte(4.5)), 0.4).where(slope_perc.gt(4.5).And(slope_perc.lte(100)), 0.5)

In [ ]:
L_factor2014 = (slope_perc.multiply(0).add(30).divide(slope_perc.multiply(0).add(22.13))).pow(m).rename("L_factor") # se multiplica por 0 y se suma 30 para crear un raster que pueda operar con la pendiente y 30 por el tamaño de pixel

#### 2.4 Inclinación de la pendiente (S)

El factor de inclinación de la pendiente representa la velocidad a la que puede fluir el agua en una superficie determinada, interactuando con el ángulo del suelo y afectando a la magnitud de la erosión del suelo.

$S = \frac{(0.43 + 0.3 * S + 0.043 * S^2)}{6.613}$

Donde *S* corresponde a la pendiente en porcentaje.

In [ ]:
S_factor2014 = slope_perc.pow(2).multiply(0.043).add(slope_perc.multiply(0.30)).add(0.43).divide(6.613)

#### 2.5 Cobertura vegetal (C)

De Jong (1994) desarrolló la siguiente relación entre los valores del factor C calibrados en campo con el índice de vegetación de diferencia normalizada (NDVI) basado en Landsat para producir una superficie continua del factor C.

$C = 0.431 - 0.805 * NDVI$

$NDVI = \frac{NIR - Red}{NIR + Red}$

Donde NIR corresponde a la reflectividad en el infrarojo cercano y Red corresponde a la reflectividad en el rojo.

In [ ]:
l8_filtered = l8.filterDate(date_from, date_to).median().clip(aoi)
ndvi = l8_filtered.normalizedDifference(['SR_B5','SR_B4']).rename("NDVI")

In [ ]:
# De Jong (1994)
C_factor2014 = ndvi.multiply(0).add(0.431).subtract(ndvi.multiply(0.805)).rename('C')

#### 2.6 Practicas control erosión (P)

El factor de práctica de control de la erosión refleja el impacto de las prácticas de apoyo sobre la tasa media anual de erosión, y también describe la relación entre las pérdidas de suelo en un campo concreto en el que se determina la práctica de control de la erosión.

En el presente ejercicio se consideró el producto de cobertura de suelos de MODIS y se asignó un valor para cada cobertura basado en Chuenchum et al., 2019.




In [ ]:
# P Factor de acuerdo a Chuenchum et al., 2019

lulc = modis.filterDate(date_from, date_to).select('LC_Type1').first().clip(aoi).rename('lulc')

# create P Facor map using an expression
P_factor2014 = lulc.expression(
                "(b('lulc') == 1) ? 0.8" +         # Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 2) ? 0.8 " +    # Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 3) ? 0.8" +     # Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
                  ": (b('lulc') == 4) ? 0.8" +     # Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
                  ": (b('lulc') == 5) ? 0.8" +     # Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 6) ? 0.8" +     # Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
                  ": (b('lulc') == 7) ? 0.8" +     # Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
                  ": (b('lulc') == 8) ? 0.8" +     # Woody Savannas: tree cover 30-60% (canopy >2m). [PRESENTE]
                  ": (b('lulc') == 9) ? 0.8" +     # Savannas: tree cover 10-30% (canopy >2m). [PRESENTE]
                  ": (b('lulc') == 10) ? 0.8" +    # Grasslands: dominated by herbaceous annuals (<2m). [PRESENTE]
                  ": (b('lulc') == 11) ? 1.0" +    # Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
                  ": (b('lulc') == 12) ? 0.5" +    # Croplands: at least 60% of area is cultivated cropland.
                  ": (b('lulc') == 13) ? 1.0" +    # Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles. [PRESENTE]
                  ": (b('lulc') == 14) ? 0.5" +    # Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
                  ": (b('lulc') == 15) ? 1.0" +    # Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
                  ": (b('lulc') == 16) ? 1.0" +    # Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
                  ": (b('lulc') == 17) ? 1.0" +    # Water Bodies: at least 60% of area is covered by permanent water bodies.
                ": 9999").rename('P').clip(aoi)    # NA

#### Modelo RUSLE

In [ ]:
soil_loss2014 = ee.Image(R_factor2014.multiply(K_factor2014).multiply(L_factor2014).multiply(S_factor2014).multiply(C_factor2014).multiply(P_factor2014).multiply(0.09)).rename("Soil Loss") # Se multiplica por 0.09 para calcular la erosión en el pixel de 30x30 m

In [ ]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = soil_loss2014

# Define the visualization parameters.
vizParams = {
  'bands': ['Soil Loss'],
  'min': 0,
  'max': 70,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

# Add legend
Map.add_colorbar(vizParams, label="Perdida de suelo (ton/ha/año)", layer_name="Perdida de suelo")

# Center the map and display the image.
Map.centerObject(aoi, 11)
Map.addLayer(image, vizParams, 'Soil loss')

# Display the map
Map

Map(center=[-35.544278357280355, -72.13937168162737], controls=(WidgetControl(options=['position', 'transparen…

## 3. Comparación antes y después de incendio

En enero de 2017 y hasta mayo de 2017 un incendio consumió más de 150.000 ha (Figura 2) en la región del Maule. Toda la cuenca del Río Purapel en Sauzal se quemó.



![](https://drive.google.com/uc?export=view&id=1P5CcQwCVhoGXpNt6uEWJybQs2o3-4JtI)
Figura 2. Incendio Las Maquinas 2017.


La figura 3 muestra la condición de la vegetación en diciembre de los años 2014 (izq.) y 2017 (der.).

![](https://drive.google.com/uc?export=view&id=15-GcrmFLhv2GIhEbRg-EqR1ssosaj-VR)
Figura 3. Condición de la vegetación en diciembre de los años 2014 (izq.) y 2017 (der.).

In [ ]:
#### Configuración ---------------------------------------------------------------------------------
date_from = '2017-01-01'
date_to = '2018-01-01'
# Poner repo GitHub
aoi = geemap.geojson_to_ee('https://github.com/LRiveroIribarne/RUSLE_Satelital_DEMO_EATS/blob/fb370ac9cfea5f5f57e7d0861487d6b935364c0b/data/aoi.geojson')

# Cargar y filtrar datos ---------------------------------------------------------------------------
pcp_filtered = pcp.select('precipitation').filterDate(date_from, date_to).sum().clip(aoi)
l8_filtered = l8.filterDate(date_from, date_to).median().clip(aoi)
ndvi = l8_filtered.normalizedDifference(['SR_B5','SR_B4']).rename("NDVI")
lulc = modis.filterDate(date_from, date_to).select('LC_Type1').first().clip(aoi).rename('lulc')

# Factor R (Cambia ya que son diferentes cantidades de precipitación) ------------------------------
R_factor2017 = ee.Image(pcp_filtered.pow(1.610).multiply(0.0483))

# Factor K (No cambia, asumimos que el suelo sigue igual) ------------------------------------------
K_factor2017 = soils_filtered.expression(
    "(b('soil') == 11) ? 0.080" +                # Sand
      ": (b('soil') == 10) ? 0.040" +            # Loamy Sand
        ": (b('soil') == 9) ? 0.300" +           # Silt
           ": (b('soil') == 8) ? 0.070" +         # Sandy Loam
            ": (b('soil') == 7) ? 0.380" +       # Silt Loam
            ": (b('soil') == 6) ? 0.300" +       # Loam
            ": (b('soil') == 5) ? 0.350" +       # Sandy Clay Loam
            ": (b('soil') == 4) ? 0.300" +       # Silty Clay Loam
            ": (b('soil') == 3) ? 0.300" +       # Clay Loam
            ": (b('soil') == 2) ? 0.220" +        # Sandy Clay
            ": (b('soil') == 1) ? 0.260" +       # Silty Clay
            ": (b('soil') == 0) ? 0.220" +       # Clay
             ": 9999").rename('K').clip(aoi)

# Factor L (No cambia, asumimos que la topografía sigue igual) -------------------------------------
L_factor2017 = (slope_perc.multiply(0).add(30).divide(slope_perc.multiply(0).add(22.13))).pow(m).rename("L_factor") # se multiplica por 0 y se suma 30 para crear un raster que pueda operar con la pendiente y 30 por el tamaño de pixel

# Factor S (No cambia, asumimos que la topografía sigue igual) -------------------------------------
S_factor2017 = slope_perc.pow(2).multiply(0.043).add(slope_perc.multiply(0.30)).add(0.43).divide(6.613)

# Factor C (Cambia ya que al cambiar la cobertura de suelo se modifica el NDVI) --------------------
C_factor2017 = ndvi.multiply(0).add(0.431).subtract(ndvi.multiply(0.805)).rename('C')

# Factor P (Cambia ya que la cobertura de suelo pasa a ser suelo desnudo tras incendio) ------------
P_factor2017 = lulc.expression(
                "(b('lulc') == 1) ? 0.8" +         # Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 2) ? 0.8 " +    # Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 3) ? 0.8" +     # Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
                  ": (b('lulc') == 4) ? 0.8" +     # Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
                  ": (b('lulc') == 5) ? 0.8" +     # Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%. [PRESENTE]
                  ": (b('lulc') == 6) ? 0.8" +     # Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
                  ": (b('lulc') == 7) ? 0.8" +     # Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
                  ": (b('lulc') == 8) ? 0.8" +     # Woody Savannas: tree cover 30-60% (canopy >2m). [PRESENTE]
                  ": (b('lulc') == 9) ? 0.8" +     # Savannas: tree cover 10-30% (canopy >2m). [PRESENTE]
                  ": (b('lulc') == 10) ? 0.8" +    # Grasslands: dominated by herbaceous annuals (<2m). [PRESENTE]
                  ": (b('lulc') == 11) ? 1.0" +    # Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
                  ": (b('lulc') == 12) ? 0.5" +    # Croplands: at least 60% of area is cultivated cropland.
                  ": (b('lulc') == 13) ? 1.0" +    # Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles. [PRESENTE]
                  ": (b('lulc') == 14) ? 0.5" +    # Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
                  ": (b('lulc') == 15) ? 1.0" +    # Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
                  ": (b('lulc') == 16) ? 1.0" +    # Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
                  ": (b('lulc') == 17) ? 1.0" +    # Water Bodies: at least 60% of area is covered by permanent water bodies.
                ": 9999").rename('P').clip(aoi)    # NA

soil_loss2017 = ee.Image(R_factor2017.multiply(K_factor2017).multiply(L_factor2017).multiply(S_factor2017).multiply(C_factor2017).multiply(P_factor2017).multiply(0.09)).rename("Soil Loss") # se multiplica para calcular el area en el pixel de 30x30 m

Downloading...
From: https://raw.githubusercontent.com/LRiveroIribarne/RUSLE_Satelital_DEMO_EATS/fb370ac9cfea5f5f57e7d0861487d6b935364c0b/data/aoi.geojson
To: /tmp/70c61f12-267b-4130-b866-ac647f3d1433.geojson
159kB [00:00, 110MB/s]                     


In [ ]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = soil_loss2017

# Define the visualization parameters.
vizParams = {
  'bands': ['Soil Loss'],
  'min': 0,
  'max': 70,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

# Add legend
Map.add_colorbar(vizParams, label="Perdida de suelo (ton/ha/año)", layer_name="Perdida de suelo")

# Center the map and display the image.
Map.centerObject(aoi, 11)
Map.addLayer(image, vizParams, 'Soil loss')

# Display the map
Map

Map(center=[-35.544278357280355, -72.13937168162737], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
vizParams = {
  'bands': ['Soil Loss'],
  'min': 0,
  'max': 70,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}

left_layer = geemap.ee_tile_layer(soil_loss2014, vizParams, 'Soil loss RUSLE 2014')
right_layer = geemap.ee_tile_layer(soil_loss2017, vizParams, 'Soil loss RUSLE 2017')

Map = geemap.Map()

Map.split_map(left_layer, right_layer)

Map.centerObject(aoi, zoom = 11)

Map.add_colorbar(vizParams, label="Perdida de suelo (ton/ha/año)", layer_name="Perdida de suelo")
Map.add_basemap('HYBRID')
Map

Map(center=[-35.544278357280355, -72.13937168162737], controls=(ZoomControl(options=['position', 'zoom_in_text…

## 4. Exportar resultados

In [ ]:
# definir directorio donde guardar los datos
dir_out = '/content/drive/MyDrive/Diplomado_suelos_UCH/Demo/resultados'

In [ ]:
geemap.ee_export_image_to_drive(R_factor2014, description = 'R_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(K_factor2014, description = 'K_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(L_factor2014, description = 'L_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(S_factor2014, description = 'S_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(C_factor2014, description = 'C_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(P_factor2014, description = 'P_factor2014', folder = dir_out, scale = 90, region=aoi.geometry())

In [ ]:
geemap.ee_export_image_to_drive(R_factor2017, description = 'R_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(K_factor2017, description = 'K_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(L_factor2017, description = 'L_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(S_factor2017, description = 'S_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(C_factor2017, description = 'C_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(P_factor2017, description = 'P_factor2017', folder = dir_out, scale = 90, region=aoi.geometry())

In [ ]:
geemap.ee_export_image_to_drive(soil_loss2014, description = 'Soil_loss2014', folder = dir_out, scale = 90, region=aoi.geometry())
geemap.ee_export_image_to_drive(soil_loss2017, description = 'Soil_loss2017', folder = dir_out, scale = 90, region=aoi.geometry())

## Referencias

1. Uddin, K., Abdul Matin, M., & Maharjan, S. (2018). Assessment of land cover change and its impact on changes in soil erosion risk in Nepal. Sustainability, 10(12), 4715.

2. Renard, K. G. (1997). Predicting soil erosion by water: a guide to conservation planning with the Revised Universal Soil Loss Equation (RUSLE). United States Government Printing.

3. Funk, C., Peterson, P., Landsfeld, M., Pedreros, D., Verdin, J., Shukla, S., ... & Michaelsen, J. (2015). The climate hazards infrared precipitation with stations—a new environmental record for monitoring extremes. Scientific data, 2(1), 1-21.

4. Boisier, J. P., Alvarez-Garretón, C., Cepeda, J., Osses, A., Vásquez, N., & Rondanelli, R. (2018, April). CR2MET: A high-resolution precipitation and temperature dataset for hydroclimatic research in Chile. In EGU General Assembly Conference Abstracts (p. 19739).

5. Zambrano, F., Wardlow, B., Tadesse, T., Lillo-Saavedra, M., & Lagos, O. (2017). Evaluating satellite-derived long-term historical precipitation datasets for drought monitoring in Chile. Atmospheric Research, 186, 26-42.

6. De Jong, S. M. (1994). Derivation of vegetative variables from a Landsat TM image for modelling soil erosion. Earth Surface Processes and Landforms, 19(2), 165-178.

7. Chuenchum, P., Xu, M., & Tang, W. (2019). Estimation of soil erosion and sediment yield in the Lancang–Mekong river using the modified revised universal soil loss equation and GIS techniques. Water, 12(1), 135.